In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import sys
import os
import os.path as osp
import cv2
from vehicle_reid.embedder import Embedder
from vehicle_reid.evaluate import Evaluator
from vehicle_reid import exp_config
from vehicle_reid.common import *
from sift_match.sift_match import *
from matplotlib import pyplot as plt
from classifier import classifier_common as clscom
import tensorflow as tf
import numpy as np
import h5py
GPU_ID = "1"
from vehicle_reid.re_ranking import re_ranking as advance_re_ranking
from track2_evaluation.eval import compute_mAP

In [2]:
#Load distance files:
test_full_ids, test_full_paths, test_full_track = read_csv_dataset(TESTFULL['csv_in'])
_, query_paths, __ = read_csv_dataset(QUERY['csv_in'])

test_bus_train = load_h5("results_dists/testfulltrainbus798x5.h5")

query_to_test_track = load_h5("results_dists/img_quetest_1052x18290.h5")
test_full_test_full = load_h5("results_dists/img_testtest_18290x18290.h5")
query_query = load_h5("results_dists/queryquery1052x1052.h5")

In [3]:
def select_top(inp_arr, selected):
    for i in np.argsort(inp_arr):
        if (i not in selected):
            selected.append(i)
            return selected, i
def select_NN(top1_view, tracklet_dists, k = 4):
    selected = [top1_view]
    a = top1_view
    for i in range(k):
        selected, b = select_top(tracklet_dists[a], selected)
        a = b
    return selected 
def re_ranking(top_list, scores_test):
    for i, track_id in enumerate(top_list):
        scores_test[track_id] = 1.0 - i * 0.05
    return scores_test

In [4]:
#BUS RERANKING
query_bus, test_bus = read_bus_query_test()
tmp = exec_query_bus(test_bus_train,
               query_to_test_track,
               test_bus,query_bus)
bus_selected = {}
for i, query_id in enumerate(query_bus):
    bus_selected[query_id] = tmp[i]

In [18]:
print(query_to_test_track.shape)
print(query_query.shape)
print(test_full_test_full.shape)

(1052, 18290)
(1052, 1052)
(18290, 18290)


In [5]:
tmp = advance_re_ranking(
    query_to_test_track,
    query_query,
    test_full_test_full,
    k1=50,
    k2=100,
    lambda_value=.5)

In [9]:
tmp = query_to_test_track.copy()
print(tmp)

[[26.94984  29.275618 25.972261 ... 25.25887  26.792643 30.789297]
 [37.312157 40.328632 37.03171  ... 39.557983 35.419235 29.78647 ]
 [28.243708 26.583248 27.195284 ... 18.520428 31.508535 30.911194]
 ...
 [29.223278 31.405657 23.798979 ... 23.956118 30.090523 30.293911]
 [32.103672 26.378838 28.403996 ... 34.27507  30.133093 37.058228]
 [28.167511 12.376411 25.023684 ... 34.196644 28.543545 38.62953 ]]


In [12]:
scores_track = 1 / tmp
final_scores = []
final_img_ids = []
top = 500
selected_tracklets = []
for i, img_query in enumerate(query_paths):
    if (i in query_bus):
        #If BUS then apply bus re-id:
        scores_track[i] = re_ranking(bus_selected[i], scores_track[i])
        selected_tracklets.append(bus_selected[i])
#     else:
# #         Apply nearest neighbor:
#         selected = select_NN(np.argmax(scores_track[i]),test_full_test_full)
#         scores_track[i] = re_ranking(selected, scores_track[i])
#         selected_tracklets.append(selected)
    #Select top 100 imgs:
#     scores_test = scores_track[i][test_full_track]
    top100 = np.argsort(scores_track[i])[::-1][:top]
    top100_scores = scores_track[i][top100]
    final_scores.append(top100_scores)
    final_img_ids.append(top100)
final_scores = np.array(final_scores)
final_img_ids = np.array(final_img_ids)

In [20]:
print(scores_track.shape)

(1052, 18290)


In [50]:
np.sort(final_scores[0])[::-1][:top].shape

(500,)

In [13]:
# SAVE SUBMISSION FILE
with open("img_track2.txt", "w") as fo:
    for i, img_query in enumerate(query_paths):
        for j in final_img_ids[i][:100]:
            img_id = int(j)+1
            fo.write("{} ".format(img_id))
        fo.write("\n")
# #EVALUATE
compute_mAP()

0.5414180468260018

In [19]:
#SAVE RESULTS TO API
result_folder = "/home/hthieu/AICityChallenge2019/test_tracklet_re_ranking_results_img_base"
gal_root = "image_test"
que_root = "image_query"
gallery_fids = test_full_paths
gallery_trklet = test_full_track

if not osp.exists(result_folder):
    os.makedirs(result_folder)
def save_result_for_query(query_name, gallery_imgs, gallery_scores, gallery_tracklets):
    gallery_imgs = [os.path.join(gal_root, x) for x in gallery_imgs]
    with open(os.path.join(result_folder, query_name.replace('.jpg','.txt')), "w") as fo:
            for i in range(top):
                fo.write("{:s},{:5f},{},{}\n".format(gallery_imgs[i],
                                                     gallery_scores[i],
                                                     True,
                                                     gallery_tracklets[i]))
#Save each query image seperately  
for i, img_query in enumerate(query_paths):
    save_result_for_query(img_query,
                         test_full_paths[final_img_ids[i]].tolist(),
                         final_scores[i],
                         test_full_track[final_img_ids[i]].tolist())
#Save index
with open(os.path.join(result_folder,"index.csv"), "w") as fo:
        for img_query in query_paths:
            query_img_path = os.path.join(que_root, img_query)
            fo.write("{},{:.5f}\n".format(query_img_path,1.0))

In [ ]:
with open("track2_tracklet.txt", "w") as fo:
    for query_trk in selected_tracklets:
        query_trk = [str(x) for x in query_trk]
        fo.write(','.join(query_trk)+"\n")